In [49]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [50]:
# Scailing
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])
test['Scaled fixed acidity'] = scaler.transform(test[['fixed acidity']])

In [51]:
#Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])

onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])

onehot = encoder.transform(test[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
test = pd.concat([test, onehot], axis = 1)
test = test.drop(columns = ['type'])

In [52]:
# Bayesian Optimization
from bayes_opt import BayesianOptimization

In [53]:
def rf_bo(max_depth, n_estimators):
    rf_params = {
        'max_depth': int(round(max_depth)),
        'n_estimators': int(round(n_estimators))
    }
    rf = RandomForestClassifier(**rf_params)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)
    
    rf.fit(X_train, y_train)
    score = accuracy_score(y_valid, rf.predict(X_valid))
    return score

In [54]:
X = train.drop(columns = ['index', 'quality'])
y = train['quality']

rf_parameter_bounds = {
    'max_depth': (1, 3),
    'n_estimators': (30, 100)
}

In [55]:
BO_rf = BayesianOptimization(f = rf_bo, pbounds = rf_parameter_bounds, random_state = 0)
BO_rf.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.5073   |  2.098    |  80.06    |
|  2        |  0.5218   |  2.206    |  68.14    |
|  3        |  0.5209   |  1.847    |  75.21    |
|  4        |  0.5164   |  1.875    |  92.42    |
|  5        |  0.5445   |  2.927    |  56.84    |
|  6        |  0.5627   |  2.604    |  52.96    |
|  7        |  0.5218   |  2.539    |  46.04    |
|  8        |  0.4782   |  1.0      |  53.91    |
|  9        |  0.5318   |  2.639    |  52.99    |
|  10       |  0.5436   |  2.657    |  53.03    |


In [56]:
max_params = BO_rf.max['params']
max_params['max_depth'] = int(max_params['max_depth'])
max_params['n_estimators'] = int(max_params['n_estimators'])
print(max_params)

{'max_depth': 2, 'n_estimators': 52}


In [57]:
BO_tuend_rf = RandomForestClassifier(**max_params)

In [66]:
# Bayesian Optimization을 이용해 XGBoost 모델 튜닝
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [67]:
# Scailing
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])
test['Scaled fixed acidity'] = scaler.transform(test[['fixed acidity']])

In [68]:
#Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])

onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])

onehot = encoder.transform(test[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
test = pd.concat([test, onehot], axis = 1)
test = test.drop(columns = ['type'])

In [69]:
from xgboost import XGBClassifier

def xgb_bo(gamma, max_depth, subsample):
    xgb_params = {
        'gamma': int(round(gamma)),
        'max_depth': int(round(max_depth)),
        'subsample': int(round(subsample))
    }
    xgb = XGBClassifier(**xgb_params)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)
    
    xgb.fit(X_train, y_train)
    score = accuracy_score(y_valid, xgb.predict(X_valid))
    return score

In [70]:
X = train.drop(columns = ['index', 'quality'])
y = train['quality']

xgb_parameter_bounds = {
    'gamma': (0, 10),
    'max_depth': (1, 3),
    'subsample': (0.5, 1)
}

BO_xgb = BayesianOptimization(f = xgb_bo, pbounds = xgb_parameter_bounds, random_state = 0)
BO_xgb.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   |   gamma   | max_depth | subsample |
-------------------------------------------------------------
[18:25:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\freshman\Desktop\AI\ai\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


|  1        |  0.5345   |  5.488    |  2.43     |  0.8014   |
[18:25:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\freshman\Desktop\AI\ai\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


|  2        |  0.55     |  5.449    |  1.847    |  0.8229   |
[18:25:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

C:\Users\freshman\Desktop\AI\ai\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


|  8        |  0.5482   |  5.411    |  2.106    |  0.8663   |
[18:25:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\freshman\Desktop\AI\ai\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


|  9        |  0.5782   |  2.303    |  2.95     |  0.7461   |
[18:25:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|  10       |  0.5409   |  3.72     |  1.231    |  0.9286   |


C:\Users\freshman\Desktop\AI\ai\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [73]:
print(BO_xgb.max['params'])

{'gamma': 2.3028884906844747, 'max_depth': 2.915034887195457, 'subsample': 0.6916244375247732}


In [74]:
xgb_tune = XGBClassifier(gamma = 2.302, max_depth = 2, subsample = 0.6916)
xgb_tune.fit(X, y)

C:\Users\freshman\Desktop\AI\ai\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:28:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=2.302, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.6916,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [75]:
pred = xgb_tune.predict(test.drop(columns = ['index']))

In [76]:
submission = pd.read_csv('data/sample_submission.csv')
submission['quality'] = pred
submission.to_csv('tune_xgb.csv', index = False)

In [77]:
# Bayesian Optimization을 이용해 LightGBM 모델 튜닝
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [78]:
# Scailing
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])
test['Scaled fixed acidity'] = scaler.transform(test[['fixed acidity']])

In [79]:
#Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])

onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])

onehot = encoder.transform(test[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
test = pd.concat([test, onehot], axis = 1)
test = test.drop(columns = ['type'])

In [81]:
from lightgbm import LGBMClassifier

def lgbm_bo(n_estimators, max_depth, subsample):
    lgbm_params = {
        'n_estimators': int(round(n_estimators)),
        'max_depth': int(round(max_depth)),
        'subsample': int(round(subsample))
    }
    lgbm = LGBMClassifier(**lgbm_params)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)
    
    lgbm.fit(X_train, y_train)
    score = accuracy_score(y_valid, lgbm.predict(X_valid))
    return score

In [82]:
X = train.drop(columns = ['index', 'quality'])
y = train['quality']

lgbm_parameter_bounds = {
    'n_estimators': (30, 100),
    'max_depth': (1, 3),
    'subsample': (0.5, 1)
}

In [83]:
BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds, random_state = 0)
BO_lgbm.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   | max_depth | n_esti... | subsample |
-------------------------------------------------------------
|  1        |  0.5618   |  2.098    |  80.06    |  0.8014   |
|  2        |  0.5691   |  2.09     |  59.66    |  0.8229   |
|  3        |  0.5973   |  1.875    |  92.42    |  0.9818   |
|  4        |  0.5764   |  1.767    |  85.42    |  0.7644   |
|  5        |  0.56     |  2.136    |  94.79    |  0.5355   |
|  6        |  0.5509   |  2.338    |  79.72    |  0.9232   |
|  7        |  0.5718   |  1.977    |  91.89    |  0.9572   |
|  8        |  0.5345   |  2.082    |  68.72    |  0.8663   |
|  9        |  0.5836   |  1.939    |  92.29    |  0.968    |
|  10       |  0.5373   |  1.744    |  38.08    |  0.9286   |


In [85]:
print(BO_lgbm.max['params'])

{'max_depth': 1.875174422525385, 'n_estimators': 92.42411005474558, 'subsample': 0.9818313802505146}


In [88]:
lgbm_tune = LGBMClassifier(n_estimators = 92, max_depth = 2, subsample = 0.9818)
lgbm_tune.fit(X, y)

LGBMClassifier(max_depth=2, n_estimators=92, subsample=0.9818)

In [89]:
pred = lgbm_tune.predict(test.drop(columns = ['index']))

In [90]:
submission = pd.read_csv('data/sample_submission.csv')
submission['quality'] = pred
submission.to_csv('tune_lgbm.csv', index = False)